# Modelling Tools - Logs and hit service
Use this notebook to:
- Hit the live service
- Access and Parse the ACI logs

In [ ]:
"""

Naming Conventions:

Package:            thispackage (short name)
Module:             this_module (short name)
Class:              ThisIsAClass
Function:           this_is_a_function
Public Method:      this_is_a_public_method
Non-Public Method:  _this_is_a_non_public_method
Variables:          thisIsAVariable
Constant:           THIS_IS_A_CONSTANT
DataFrame:          dfThisIsADataFrame
List:               lsThisIsAList
Array:              arThisIsAnArray
Series:             srThisIsASeries

"""
__status__ = "development"
__version__ = '1.0.0'
__date__ = "26 Oct 2020"
__author__ = 'Laurence Day,Chris Pickford,Laurence Day'

In [ ]:
###########################################################
###########################################################

# What github repository are you using?

project = "github repo name"

# Set the service name before continuing

serviceName = ''

# What is the name of the model as used in the deployment json and in score.py?

model_name = ""

# What subscription is this in 

deploymentEnvironment = ''



###########################################################
###########################################################

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize 
import pandas as pd
import azureml.core
import os
import pickle
import sys
import json
import random
import requests

from azureml.core import Workspace, Webservice
from azureml.core.model import Model


# display the core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

## Specify subscription and model name

In [ ]:
### CHANGE TO YOUR WORKSPACE

ws = Workspace(
    workspace_name=workspace_name, 
    subscription_id=subscription_id, 
    resource_group=resource_group)
display(ws)


print('Referencing:', deploymentEnvironment)




In [ ]:
ws

In [ ]:
serviceName

In [ ]:
service = Webservice(ws, serviceName)
service

# Step 1: Hit Service

In [ ]:
def hit_service(TestData_json):
    import requests
    key, secondary = service.get_keys() 
    headers = {'Content-Type':'application/json',  'Authorization':('Bearer '+ key)}
    resp = requests.post(service.scoring_uri, TestData_json, headers=headers)
    print(resp)
    print(resp.text)
    #print(resp.json())

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating ):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)      

## Setup Payload

### Generate manual simulated payload
Each payload is hardcoded to give a set outcome

In [ ]:
def generate_payload():
    lsTestData = []
    jsonPayload = {
        
    }

    jsonPayload = json.dumps(jsonPayload, cls=NpEncoder)
    jsonTest = json.loads(jsonPayload)   
    return jsonTest

### Generate payload from normal distributions

In [ ]:
def payload_chance(chance):
    
    if random.random() >=1-chance:
        return True
    else:
        return False
    
def generate_json(inputs, uniqueIdentifier):
    
    master = {
    "1": uniqueIdentifier,
    "2" : np.random.normal(6000, 1000, 1)[0],
    "3" : np.random.normal(35, 7, 1)[0],
    "4" : np.random.normal(1500, 700, 1)[0],
    "5" : np.random.normal(6000, 1000, 1)[0],
    "6" : "x",
    "7": 1,
    "8": "y",
    "9": "z",
    "10" : {"score": np.random.normal(400, 100, 1)[0]}
    }
    
    payload = {}
    for component in inputs:
        payload[component] = master[component]
    payload['uniqueIdentifier'] = uniqueIdentifier
    payload = json.dumps(payload, cls=NpEncoder)
    return json.loads(payload)
    

def create_payload(id):
    
    colsOfInterest = ["","","","","","","","",""]
    if payload_chance(.9):
        print('Good payload')
        return generate_json(colsOfInterest,id)
    else:
        if payload_chance(.6):
            print('warning payload')
            randCol = random.choice(colsOfInterest)
            colsOfInterest.remove(random.choice(colsOfInterest))
            return generate_json(colsOfInterest,id)
        else:
            print('hard fail payload')
            return generate_json([],id)


### Hit the service with simulated data

In [ ]:
for i in range(1,2,1):        
    jsonPayload = create_payload(i)
    #print(jsonPayload)
    hit_service(json.dumps(jsonPayload, cls=NpEncoder))

# Step 2: Obtain and Parse Logs

In [ ]:
import re
import ast
import json
from IPython.display import display

### Obtain payload hitting the container

In [ ]:
logs = service.get_logs(num_lines=99900000)
lslogs = logs.split('\n')
foundTimeStamp =False

lsPayloads = []
i = 0
for log in lslogs: 
    try:
        if ' - - [' in log:
            #print(log)
            timestamp = re.search('127.0.0.1 - - \[(.*) \+',log).group(1)
            #print(timestamp)
            foundTimeStamp = True
            
        if 'Received input' in log and foundTimeStamp: # or '/score' in log: 
            #print(timestamp)
            log = log.replace('Received input: ', '')
            loadedLog = json.loads(log)
            loadedLog['timeStamp'] = timestamp
            lsPayloads.append(loadedLog)
            foundTimeStamp = False
            
        
    except:
        print(log)
        #pass
    i+=1
    
dfPayloads = pd.DataFrame.from_records(lsPayloads)
dfPayloads['timeStamp'] = pd.to_datetime(dfLogs['timeStamp'],format = '%d/%b/%Y:%H:%M:%S',dayfirst=False,yearfirst=False)

display(dfLogs)

In [ ]:
# dfPayloads.to_csv('OriginalPayload.csv')

### Obtain logs writen by Score.py

In [ ]:
logs = service.get_logs(num_lines=99900000)
lslogs = logs.split('\n')
lsOutcomes = []
i = 0
for log in lslogs: 
    if ' - - [' in log:
        #print(log)
        timestamp = re.search('127.0.0.1 - - \[(.*) \+',log).group(1)
        #print(timestamp)
        
    if '"ApplicationId"' in log and 'Received input' not in log: # or '/score' in log: 
        #print(log, i)
        loadedLog = json.loads(log)
        loadedLog['timeStamp'] = timestamp
        lsOutcomes.append(loadedLog)
        #print(log_to_df)
    
        
    i+=1
dfLogs = pd.DataFrame.from_records(lsOutcomes)
dfLogs['timeStamp'] = pd.to_datetime(dfLogs['timeStamp'],format = '%d/%b/%Y:%H:%M:%S',dayfirst=False,yearfirst=False)
dfLogs = dfLogs[['timeStamp','Service','Severity']].join(pd.DataFrame.from_dict([l for l in dfLogs['Message']if type(l) is dict]))
dfLogs['ApplicationId'] = dfLogs['ApplicationId'].str.replace('[','').str.replace(']','')#.astype(int)

display(dfLogs)

### Look at raw service logs

In [ ]:
import pprint as pp
pp.pprint(service.get_logs())